# Pathway Analysis

The main purpose of Pathway Analysis tools is to analyze data obtained from high-throughput technologies, detecting relevant groups of related genes that are altered in case samples in comparison to a control. In this manner, Pathway Analysis methods seek to overcome the problem of interpreting overwhelmingly large lists of important, but isolated genes detached of biological context, which are the main output of most basic high-throughput data analysis, as differential expression analysis. A standard approach to addressing this problem is pathway enrichment analysis, which summarizes the large gene list as a smaller list of more easily interpretable pathways. Pathways are statistically tested for over-representation in the experimental gene list relative to what is expected by chance, using several common statistical tests that consider the number of genes detected in the experiment, their relative ranking and the number of genes annotated to a pathway of interest.

A principled way to interpret such gene sets is to compare them with a database of well-annotated gene sets, such as biological pathways. For example, one of the most widely used approach, Over-representation Analysis (ORA), counts the number of common genes shared by an input gene set and each annotated gene set, and applies a statistical test, such as the cumulative hyper-geometric test, to calculate the statistical significance of the overlap. A p-value cutoff, e.g. 0.05, is then applied to select the annotated gene sets that have statistically significant overlaps with the input gene set. An extension of ORA known as the Gene Set Enrichment Analysis (GSEA) tries to eliminate the need for an ad hoc cutoff (e.g. expression fold change) that is often used in defining the input gene set. GSEA works by ranking all genes in the genome according to, say, level of differential expression, and tests whether any annotated gene set is ranked unexpected high or low through running-sum statistic.

## Learning Objectives:
1. Introduction to enrichment analysis
3. Downloading terms, pathway gene set from GO and KEGG.
4. Saving results to GMT file format.

## Enrichment analysis
 Gene Set Enrichment Analysis methods help gain insight into obtained gene lists by identifying pathways that are enriched in a gene list more than would be expected by chance. The three major steps involved the process include; definition of a gene list from omics data, determination of statistically enriched pathways, and visualization and interpretation of the results.

## Enrichment analysis using ORA
Over-representation (or enrichment) analysis is a statistical method that determines whether genes from pre-defined sets (ex: those belonging to a specific GO term or KEGG pathway) are present more than would be expected (over-represented) in a subset of your data.



The p-value can be calculated by hypergeometric distribution.

![](./images/Module4/ora_p_value.png)


where N is the total number of genes in the background distribution, M is the number of genes within that distribution that are annotated (either directly or indirectly) to the gene set of interest, n is the size of the list of genes of interest and k is the number of genes within that list which are annotated to the gene set. The background distribution by default is all the genes that have annotation.

## Over Representation Analysis Using GO

library(tidyverse)


In [1]:
data <- read_csv("./data/DE_genes.csv")


ERROR: Error in read_csv("./data/DE_genes.csv"): could not find function "read_csv"


In [ ]:
data.head()

In [ ]:
mask <- data$adj.P.Val < 0.05 &
        abs(data$logFC) > log2(2)

In [ ]:
deGenes <- rownames(data[mask,])


In [ ]:
deGenes

In [ ]:
length(deGenes)


In [ ]:
geneUniverse <- rownames(data)


In [ ]:
length(geneUniverse)


In order to assess functional enrichment, both DE gene list and gene universe must be annotated in Entrez IDs:


In [ ]:
library(org.Hs.eg.db)


In [ ]:
deGenes <- unlist(mget(deGenes, envir=org.Hs.egENSEMBL2EG,
                       ifnotfound = NA))


In [ ]:
geneUniverse <- unlist(mget(geneUniverse, envir=org.Hs.egENSEMBL2EG,
                            ifnotfound = NA))


The GO enrichment analysis using clusterProfiler is performed by


In [ ]:
library(clusterProfiler)


In [ ]:
ans.go <- enrichGO(gene = deGenes, ont = "BP",
                   OrgDb ="org.Hs.eg.db",
                   universe = geneUniverse,
                   readable=TRUE,
                   pvalueCutoff = 0.05)

In [ ]:
tab.go <- as.data.frame(ans.go)


In [ ]:
tab.go<- subset(tab.go, Count>5)


In [ ]:
tab.go[1:5, 1:6]


## Visualization


All analyses performed with clusterProfiler can be visualized by different plots:


In [ ]:
library(enrichplot)


In [ ]:
p1 <- barplot(ans.dis, showCategory=10)


In [ ]:
p1

In [ ]:
p2 <- dotplot(ans.kegg, showCategory=20) + ggtitle("KEGG")
p3 <- dotplot(ans.dis, showCategory=20) + ggtitle("Disease")


In [ ]:
plot_grid(p2, p3, nrow=2)


In [ ]:
p4 <- upsetplot(ans.dis)


In [ ]:
p4

In [ ]:
p5 <- emapplot(ans.kegg)


In [ ]:
p5

In [ ]:
 install.packages("cowplot")

In [ ]:
cowplot::plot_grid(p1, p3, p5, ncol=2, labels=LETTERS[1:3])


## Enrichment Analysis using FGSEA


This submodule describes FGSEA, one of the tools for evaluating pathway enrichment in transcriptional data and it stands for Fast preranked Gene Set Enrichment Analysis (GSEA). It can quickly and accurately calculate arbitrarily low GSEA P-values for a collection of gene sets based on an adaptive multi-level split Monte-Carlo scheme.

 Typical experimental design consists in comparing two conditions with several replicates using a differential gene expression test followed by preranked GSEA performed against a collection of hundreds and thousands of pathways. However, the reference implementation of this method cannot accurately estimate small P-values, which significantly limits its sensitivity due to multiple hypotheses correction procedure. FGSEA, on the other hand, is able to estimate arbitrarily low GSEA P-values with a high accuracy in a matter of minutes or even seconds.


## GSEA Enrichment Analysis
# Inputs:
gene_list = Ranked gene list ( numeric vector, names of vector should be gene names)
GO_file= Path to the “gmt” GO file on your system.
pval = P-value threshold for returning results


# Steps
1. Run GSEA (package: fgsea)
2. Collapse redundant GO terms using a permutation test
3. Return GSEA plot and data.frame of results

# FGSEA Using GO

In [ ]:
library(tidyverse)


In [ ]:
data <- read_csv("./data/DE_genes.csv")


In [ ]:
data

In [ ]:
mask <- data$adj.P.Val < 0.05 &
  abs(data$logFC) > log2(2)

In [ ]:
deGenes <- rownames(data[mask,])


In [ ]:
deGenes

In [ ]:
length(deGenes)


In [ ]:
geneUniverse <- rownames(data)


In [ ]:
length(geneUniverse)


## The FGSEA Analysis Procedure (from the author)
Loading Libraries


In [ ]:
library(data.table)


In [ ]:
library(fgsea)


In [ ]:
library(ggplot2)


Loading example pathways and gene-level statistics:


In [ ]:
data(examplePathways)


In [ ]:
data(exampleRanks)


#fgsea has a default lower bound eps=1e-10 for estimating P-values. If you need to estimate P-value more accurately, you can set the eps argument to zero in the fgsea function.


In [ ]:
fgseaRes <- fgsea(pathways = examplePathways,
                  stats    = exampleRanks,
                  eps      = 0.0,
                  minSize  = 15,
                  maxSize  = 500)

In [ ]:
head(fgseaRes[order(pval), ])


# One can make an enrichment plot for a pathway:


In [ ]:
plotEnrichment(examplePathways[["5991130_Programmed_Cell_Death"]],
               exampleRanks) + labs(title="Programmed Cell Death")


# Or make a table plot for a bunch of selected pathways:


In [ ]:
topPathwaysUp <- fgseaRes[ES > 0][head(order(pval), n=10), pathway]


In [ ]:
topPathwaysDown <- fgseaRes[ES < 0][head(order(pval), n=10), pathway]


In [ ]:
topPathways <- c(topPathwaysUp, rev(topPathwaysDown))


In [ ]:
plotGseaTable(examplePathways[topPathways], exampleRanks, fgseaRes,
              gseaParam=0.5)

## FGSEA Enrichment Analysis With KEGG Data

In [ ]:
# To load the GMT file for enrichment analysis, we can use GSA.read.gmt function available in the GSA R package. Here is the code to install the package and load the GMT file

suppressMessages({
  suppressWarnings(install.packages("GSA"))
  suppressPackageStartupMessages({
    suppressWarnings(library(GSA))
  })
})

In [ ]:
# Load the GMT file from disk, we use "invisible" function to supress the excessive output message from the "GSA.read.gmt" function

invisible(capture.output(pathways <- GSA::GSA.read.gmt("./data/KEGG_pathways.gmt")))

In [ ]:
# View first five pathways and related gene sets
pathways$genesets[1:5]

In [ ]:
# View the name of the pathways
pathways$geneset.names[1:5]

In [ ]:
# View the description of each pathway
pathways$geneset.descriptions[1:5]

# FGSEA Using KEGG

# To load the GMT file for enrichment analysis, we can use GSA.read.gmt function available in the GSA R package. Here is the code to install the package and load the GMT file

suppressMessages({
  suppressWarnings(install.packages("GSA"))
  suppressPackageStartupMessages({
    suppressWarnings(library(GSA))
  })
})


# Load the GMT file from disk, we use "invisible" function to supress the excessive output message from the "GSA.read.gmt" function

invisible(capture.output(pathways <- GSA::GSA.read.gmt("./data/KEGG_pathways.gmt")))
pathways
# View first five pathways and related gene sets
pathways$genesets[1:5]

# View the name of the pathways
pathways$geneset.names[1:5]

# View the description of each pathway
pathways$geneset.descriptions[1:5]

#//GeneList
DE <- readRDS("./data/DE_genes.rds")

# Get p-value from DE results
gene_list <- DE$adj.P.Val
#// Gene List
gene_list
GSEA = function(gene_list, pathways, pval) {
  set.seed(54321)

  library(dplyr)
  library(fgsea)
  if ( any( duplicated(names(gene_list)) )  ) {
    warning("Duplicates in gene names")
    gene_list = gene_list[!duplicated(names(gene_list))]
  }
  if  ( !all( order(gene_list, decreasing = TRUE) == 1:length(gene_list)) ){
    warning("Gene list not sorted")
    gene_list = sort(gene_list, decreasing = TRUE)
  }

  myGO = fgsea::gmtPathways(pathways)

  fgRes <- fgsea::fgsea(pathways = myGO,
                        stats = gene_list,
                        minSize=15, ## minimum gene set size
                        maxSize=400, ## maximum gene set size
                        nperm=10000) %>%
    as.data.frame() %>%
    dplyr::filter(padj < !!pval) %>%
    arrange(desc(NES))
  message(paste("Number of signficant gene sets =", nrow(fgRes)))


  message("Collapsing Pathways -----")
  concise_pathways = collapsePathways(data.table::as.data.table(fgRes),
                                      pathways = myGO,
                                      stats = gene_list)
  fgRes = fgRes[fgRes$pathway %in% concise_pathways$mainPathways, ]
  message(paste("Number of gene sets after collapsing =", nrow(fgRes)))

  fgRes$Enrichment = ifelse(fgRes$NES > 0, "Up-regulated", "Down-regulated")
  filtRes = rbind(head(fgRes, n = 10),
                  tail(fgRes, n = 10 ))


  total_up = sum(fgRes$Enrichment == "Up-regulated")
  total_down = sum(fgRes$Enrichment == "Down-regulated")
  header = paste0("Top 10 (Total pathways: Up=", total_up,", Down=",    total_down, ")")


  colos = setNames(c("firebrick2", "dodgerblue2"),
                   c("Up-regulated", "Down-regulated"))


  g1= ggplot(filtRes, aes(reorder(pathway, NES), NES)) +
    geom_point( aes(fill = Enrichment, size = size), shape=21) +
    scale_fill_manual(values = colos ) +
    scale_size_continuous(range = c(2,10)) +
    geom_hline(yintercept = 0) +
    coord_flip() +
    labs(x="Pathway", y="Normalized Enrichment Score",
         title=header) +
    th

  output = list("Results" = fgRes, "Plot" = g1)
  return(output)
}



In [ ]:
## Enrichment analysis using gsa


In [ ]:
# // GSA using GO

In [ ]:
# // GSA with KEGG